In [1]:
import numpy as np
import pandas as pd
from datetime import date, timedelta, datetime

In [2]:
"""
Spot index levels:
- Use the index closing price on the valuation date (e.g., SPX = 6600.35, NDX = 24223.69).
- This serves as the starting point (S) for futures fair value.

Dividend yield (q):
- Indexes (SPX, NDX) do not publish dividend yields directly.
- We use ETF proxies:
    * SPY dividend yield (~1.11%) → proxy for SPX
    * QQQ dividend yield (~0.47%) → proxy for NDX
- These represent the expected dividends paid out over time, which reduce futures fair value.

Risk-free rate (r):
- Taken from zero-coupon Treasury STRIPS yields (Mid Yield).
- For each futures expiry, choose the STRIP maturity ON or AFTER that expiry date.

Formula:
- Futures fair value is computed with cost-of-carry:
    F = S * exp((r - q) * T)
  where T is the time to expiry in years (ACT/365).
  
"""

S_SPX = 6600.35
S_NDX = 24223.69

q_SPX = 1.11 / 100.0   # SPY dividend yield as decimal
q_NDX = 0.47 / 100.0   # QQQ dividend yield as decimal 

In [3]:
# Import zero coupon treasury principal strips  - data from Bloomberg as of 24 Sep 2025
treasury_principal_strips_df = pd.read_excel(rf"C:\Users\anjal\Desktop\Fall 2025\Derivatives\Homework\Derivatives\HW2\treasury_principal_strips.xlsx")

# Drop rows without a usable date or yield
treasury_principal_strips_df = treasury_principal_strips_df.dropna(subset=["Maturity Date", "Yield"]).sort_values(["Maturity Date"]).reset_index(drop=True)
treasury_principal_strips_df

,Maturity Date,Yield
0,2026-05-15,0.03750
1,2026-08-15,0.03582
2,2026-11-15,0.03396
3,2027-02-15,0.03411
4,2027-05-15,0.03641
5,2027-08-15,0.03581
6,2027-11-15,0.03548
7,2028-02-15,0.03601
8,2028-05-15,0.03599
9,2028-08-15,0.03551


In [4]:
treasury_principal_strips_df.dtypes
# Columns have the right datatypes

Maturity Date    datetime64[ns]
Yield                   float64
dtype: object

In [5]:
# -------------------------------------------------------------------
# Equity index futures expiry convention (CME Globex):
# - Standard contracts (SPX, NDX, etc.) expire on the THIRD FRIDAY
#   of March, June, September, and December (the IMM cycle).
# - This convention avoids clashes with month-end, holidays, and
#   provides consistency for traders and hedgers.

# Why we need this function:
# - Given a year and month (e.g., Dec 2025), we want to calculate
#   the exact expiry date of the futures contract.
# - The code:
#   1) Starts at the 1st of the month.
#   2) Finds the first Friday.
#   3) Adds two more weeks → the 3rd Friday.

# Example:
#   third_friday(2025, 12) → 19-Dec-2025
#   third_friday(2026,  3) → 20-Mar-2026
#
# This ensures our pricing model uses the ACTUAL expiry dates
# instead of an approximation (like month-end).
# -------------------------------------------------------------------

def get_third_friday(year, month):
    # 1. Start at the 1st day of the given month
    d0 = date(year, month, 1)

    # 2. Find the first Friday of that month
    # weekday(): Monday=0, Tuesday=1, ..., Friday=4
    # (4 - d0.weekday()) % 7 gives "how many days to add" to reach the first Friday
    first_friday = d0 + timedelta(days=(4 - d0.weekday()) % 7)

    # 3. Add 2 more weeks (14 days) to move from the 1st Friday → 3rd Friday
    third_friday = first_friday + timedelta(weeks=2)

    # 4. Return the 3rd Friday as a date object
    return third_friday

In [ ]:
# -------------------------------------------------------------------
# Why do we need this function?
# - Futures expiry dates (3rd Fridays) rarely match the STRIPS
#   maturity dates exactly.
# - To price fairly, we need a risk-free rate (r) that is valid
#   through the entire life of the futures contract.

# What this function does:
# - Takes a target expiry date (e.g., 19-Dec-2025).
# - Looks through the STRIPS curve for the first maturity date
#   ON or AFTER that expiry.
# - Returns two things:
#     1) The yield of that STRIP (as decimal) → used as r.
#     2) The STRIP maturity date chosen (for reporting).

# Why "on or after"?
# - Ensures the rate we use extends beyond expiry (not before).
# - If no later STRIP exists, we use the last available maturity.

# Example:
#   Expiry = 19-Dec-2025
#   Available STRIPS maturities = [2026-05-15, 2026-08-15, 2026-11-15]
#   → Picks 2026-05-15 and its yield.
# -------------------------------------------------------------------

def get_rate_on_or_after(strips: pd.DataFrame, target: date) -> tuple[float, date]:
    # 1. Filter STRIPS to only those maturing ON or AFTER the futures expiry
    # Example: if expiry = 19-Dec-2025, we want STRIPS with maturity >= 19-Dec-2025
    after = strips[pd.to_datetime(strips[date_col]).dt.date >= target]

    # 2. If we found at least one STRIP, take the first one (earliest maturity after expiry)
    #    If not (expiry goes beyond last STRIP), fall back to the last available STRIP
    row = after.iloc[[0]] if not after.empty else strips.iloc[[-1]]

    # 3. Extract the yield value (as a float)
    r = float(row[yield_col].values[0])

    # 4. Extract the maturity date of that STRIP (as a Python date object)
    mat = pd.to_datetime(row[date_col].values[0]).date()

    # 5. Return both: (risk-free rate, STRIP maturity date used)
    return r, mat

In [7]:
def fair_value(S: float, r: float, q: float, start: date, expiry: date) -> tuple[float, float]:
    # 1. Compute time to expiry (T) in years
    # Using ACT/365: actual days between valuation date and expiry / 365
    T = (expiry - start).days / 365.0  

    # 2. Apply cost-of-carry model with continuous compounding:
    #    Futures price = Spot * exp((r - q) * T)
    #    - r = risk-free rate
    #    - q = dividend yield
    #    - (r - q) = net cost of carry
    F = S * np.exp((r - q) * T)

    # 3. Return both the fair futures price and time to expiry
    return F, T

In [13]:
targets = [
    ("Dec25", 2025, 12),
    ("Mar26", 2026,  3),
    ("Jun26", 2026,  6),
    ("Sep26", 2026,  9),
    ("Dec26", 2026, 12),
]

expiries = [(lbl, get_third_friday(y, m)) for lbl, y, m in targets]
df_expiries = pd.DataFrame(expiries, columns=["Contract", "Expiry Date"])
today = date.today()

In [19]:
rows = [{
    "Contract":"Spot",
    "Expiry Date": today.strftime("%d-%b-%Y"),
    "STRIPS Maturity Used": "",
    "SpotRate(STRIPS)": "",
    "T_years": "0.000",
    "SPX Fair Value": f"{S_SPX:,.2f}",
    "NDX Fair Value": f"{S_NDX:,.2f}"
}]

date_col  = "Maturity Date"
yield_col = "Yield" 

for _, r in df_expiries.iterrows():
    lbl, exp = r["Contract"], r["Expiry Date"]
    rf, rf_mat = get_rate_on_or_after(treasury_principal_strips_df, exp)
    F_spx, T = fair_value(S_SPX, rf, q_SPX, today, exp)
    F_ndx, _ = fair_value(S_NDX, rf, q_NDX, today, exp)
    rows.append({
        "Contract": lbl,
        "Expiry Date": exp.strftime("%d-%b-%Y"),
        "STRIPS Maturity Used": rf_mat.strftime("%d-%b-%Y"),
        "SpotRate(STRIPS)": f"{rf*100:.3f}%",
        "T_years": f"{T:.3f}",
        "SPX Fair Value": f"{F_spx:,.2f}",
        "NDX Fair Value": f"{F_ndx:,.2f}"
    })

fair_value_df = pd.DataFrame(rows)

In [20]:
fair_value_df

,Contract,Expiry Date,STRIPS Maturity Used,SpotRate(STRIPS),T_years,SPX Fair Value,NDX Fair Value
0,Spot,29-Sep-2025,,,0.000,"6,600.35","24,223.69"
1,Dec25,19-Dec-2025,15-May-2026,3.750%,0.222,"6,639.13","24,400.66"
2,Mar26,20-Mar-2026,15-May-2026,3.750%,0.471,"6,682.97","24,601.01"
3,Jun26,19-Jun-2026,15-Aug-2026,3.582%,0.721,"6,718.97","24,773.00"
4,Sep26,18-Sep-2026,15-Nov-2026,3.396%,0.970,"6,748.32","24,920.96"
5,Dec26,18-Dec-2026,15-Feb-2027,3.411%,1.219,"6,788.13","25,108.01"
